# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import City Data
city_weather_data = "Exported City Weather Data.csv"
city_weather_data_path = "../WeatherPy/city_data/Exported City Weather Data.csv"
city_weather_data = pd.read_csv(city_weather_data_path)
city_weather_data

,Unnamed: 0,Country,City,Latitude,Longitude,Current Temp(c),Max Temp(c),Humidity (%),Cloudiness,Windspeed (km/h)
0,0,IT,Quartucciu,39.2519,9.1794,13.89,13.89,67,0,7.416
1,1,RO,Valea Ursului,46.8167,27.0833,13.16,13.29,73,99,14.544
2,2,US,Owings Mills,39.4196,-76.7803,20.42,22.97,57,7,1.620
3,3,BR,Barra,-11.0894,-43.1417,28.47,28.47,51,65,22.860
4,4,MD,Soroca,48.1558,28.2975,13.56,13.56,78,28,12.780
...,...,...,...,...,...,...,...,...,...,...
495,495,ZA,Bredasdorp,-34.5322,20.0403,14.23,14.23,73,78,8.928
496,496,CR,Alajuela,10.0167,-84.2167,21.90,24.45,93,75,14.832
497,497,LK,Polonnaruwa,7.9333,81.0000,24.21,24.21,93,31,3.744
498,498,PE,San Jerónimo,-11.9472,-75.2829,11.83,11.83,72,87,11.916


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store Variables for Lat and Long
City = city_weather_data[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = city_weather_data["Humidity (%)"].astype(float)

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

gmaps.figure(layout=figure_layout)
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(City, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create a new dataframe and export data.
idealconditions = city_weather_data
idealconditions_df = idealconditions[idealconditions["Max Temp(c)"].between(21, 27)].dropna()
idealconditions_df = idealconditions_df[idealconditions_df["Windspeed (km/h)"].between(0, 15)].dropna()
idealconditions_df = idealconditions_df[idealconditions_df["Cloudiness"].between(0, 0)].dropna()

idealconditions_df.to_csv("Data Exports\idealconditions.csv")
idealconditions_df

,Unnamed: 0,Country,City,Latitude,Longitude,Current Temp(c),Max Temp(c),Humidity (%),Cloudiness,Windspeed (km/h)
105,105,US,Yucaipa,34.0336,-117.0431,23.75,26.76,19,0,3.204
244,244,US,Radford,37.1318,-80.5764,24.73,26.51,39,0,14.832
277,277,IN,Khudāganj,28.1500,79.7167,26.78,26.78,20,0,11.124
351,351,IN,Thākurganj,26.4500,88.1333,24.57,24.57,61,0,12.204
418,418,BR,Castro Alves,-12.7656,-39.4283,21.73,21.73,92,0,7.668
461,461,JP,Kodaira,35.7264,139.4839,18.30,21.29,85,0,5.544
466,466,US,Medford,42.3265,-122.8756,22.48,23.67,34,0,7.416


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# Create new DF with a new "Hotel Name" column
hotel_df = idealconditions_df
hotel_df["Hotel Name"] = ""

# Create the base url and define parameters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get lat and long from df
    lat = row["Latitude"]
    long = row["Longitude"]
    params["location"] = f"{lat}, {long}"

    # assemble url and make API request
    print(f"Retrieving closest hotel for Index {index}: {lat, long}.")
    response = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]['name']
    
    except:
    #(KeyError, IndexError):
       print("Missing field/result... skipping.")


Retrieving closest hotel for Index 105: (34.0336, -117.0431).
Retrieving closest hotel for Index 244: (37.1318, -80.5764).
Retrieving closest hotel for Index 277: (28.15, 79.7167).
Retrieving closest hotel for Index 351: (26.45, 88.1333).
Retrieving closest hotel for Index 418: (-12.7656, -39.4283).
Retrieving closest hotel for Index 461: (35.7264, 139.4839).
Retrieving closest hotel for Index 466: (42.3265, -122.8756).


In [11]:
# Data to check the hotel name is stored...

hotel_df

,Unnamed: 0,Country,City,Latitude,Longitude,Current Temp(c),Max Temp(c),Humidity (%),Cloudiness,Windspeed (km/h),Hotel Name
105,105,US,Yucaipa,34.0336,-117.0431,23.75,26.76,19,0,3.204,Yucaipa Regional Park
244,244,US,Radford,37.1318,-80.5764,24.73,26.51,39,0,14.832,La Quinta Inn by Wyndham Radford
277,277,IN,Khudāganj,28.1500,79.7167,26.78,26.78,20,0,11.124,Lallu singh
351,351,IN,Thākurganj,26.4500,88.1333,24.57,24.57,61,0,12.204,Happy Nayak
418,418,BR,Castro Alves,-12.7656,-39.4283,21.73,21.73,92,0,7.668,Hotel e Restaurante do Poeta
461,461,JP,Kodaira,35.7264,139.4839,18.30,21.29,85,0,5.544,Issa Annex
466,466,US,Medford,42.3265,-122.8756,22.48,23.67,34,0,7.416,Hampton Inn Medford


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Country</dt><dd>{Country}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Country", "City", "Latitude", "Longitude", "Hotel Name"]]

In [15]:
# Customize the size of the figure
figure_layout = {
    'width': '500px', 
    'height': '500px', 
    'border': '1px solid black', 
    'padding': '1px', 'margin': '0 auto 0 auto'
}

gmaps.figure(layout=figure_layout)

# Create a map using state centroid coordinates to set markers
marker_locations = locations[["Latitude", "Longitude"]]

# 1. Create heat layer
heat_layer = gmaps.heatmap_layer(City, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# 2. Create a marker_layer
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)

# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Add layers
fig.add_layer(heat_layer)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))